# Эксперимент №1: расчет характеристик тандемной сети с узлами MAP/PH/1/N с помощью имитационного моделирования и машинного обучения

В этом эксперименте мы рассчитаем различные характеристики сетей с линейной топологией, на вход которых поступает MAP-поток, а обслуживание имеет распределение фазового типа. Сначала мы рассчитаем характеристики на заданной сетке статистических параметров с помощью имитационного моделирования сети, а затем используем полученные результаты для обучения нейросетевых и других моделей ML, которые сможем использовать для очень быстрой оценки характеристик сетей. Например, такой подход полезен при нахождении решений задач оптимизации топологии, когда характеристики сетей с линейной топологией являются ограничениями в алгоритме ветвей и границ. 

## Подготовка ноутбука

Импортируем все необходимые библиотеки, настроим флаги и константы, которыми будем пользоваться в эксперименте, и определим служебные функции, чтобы не отвлекаться на них в расчетной части эксперимента.

In [1]:
# Если что-то меняется в коде, мы хотим это сразу видеть здесь
%load_ext autoreload
%autoreload 2

In [2]:
import os

import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm

from tabulate import tabulate
from tqdm.notebook import tqdm

from itertools import product
from collections import namedtuple
import time
import numpy as np
import pandas as pd

# Подключаем локальные пакеты
from pyqumo.random import Distribution, Exponential, HyperExponential, Erlang
from pyqumo.cqumo.sim import simulate_tandem

In [3]:
# Настраиваем matplotlib
%matplotlib inline
matplotlib.rcParams.update({'font.size': 16})

Определим переменные окружения, которые будут использоваться в эксперименте.

In [4]:
# Нужно ли пересчитывать все, или можно использовать результаты из файлов
FORCE_SIMULATION = False
SIM_FILE_NAME = '01_tandem_simulation.csv'
SIM_FILE_DIR = 'data'
SIM_FILE_PATH = os.path.join(SIM_FILE_DIR, SIM_FILE_NAME)

# Зададим число пакетов, передачу которых по сети мы будем моделировать.
# Чем выше это число, тем точнее результаты, но на их получение нужно больше времени.
NUM_PACKETS = 100000

# Цветовая схема для графиков
CMAP_NAME = 'viridis'

In [5]:
def get_color(x):
    """
    Получить цвет из текущей карты.
    """
    return cm.get_cmap(CMAP_NAME)(x)

## Нахождение PH-распределений

PH-распределения для моделирования входящего потока и времени обслуживания будем находить по первым двум моментам по следующим правилам:

- если коэффициент вариации $c = \sigma / m_1 < 1$, то в качестве PH-распределения возьмем распределение Эрланга с самым близким значением $\tilde{\sigma}$;
- если коэффициент вариации $c > 1$, то в качестве PH-распределения возьмем гиперэкспоненциальное распределение;
- если коэффициент вариации $c = 1$, то распределение - экспоненциальное.

In [6]:
def fit_ph(avg: float, std: float) -> Distribution:
    """
    Возвращает PH-распределение по стандартному отклонению и среднему значению.
    """
    cv = std / avg
    if cv == 1:
        return Exponential(avg)
    if cv > 1:
        return HyperExponential.fit(avg, std)
    return Erlang.fit(avg, std)

## Вызов имитационной модели, чтение и сохранение результатов

Определим полезные функции, которые нам потребуются для работы с данными имитационного моделирования:

- `load_sim_data()`: читает файл `SIM_FILE_NAME`, если он существует, или создает новый `DataFrame` для хранения данных о результатах имитационного моделирования.
- `save_sim_data()`: сохраняет результаты, записанные в `DataFrame`, в файл `SIM_FILE_PATH`.

In [7]:
COLUMNS = (
    'ArrAvg', 
    'ArrStd', 
    'ArrCv', 
    'SrvAvg', 
    'SrvStd', 
    'SrvCv', 
    'Rho', 
    'NetSize', 
    'Capacity', 
    'NumPackets',
    'DelayAvg', 
    'DelayStd', 
    'DeliveryProb',
)


def save_sim_data(df: pd.DataFrame):
    """
    Сохранить в файл данные о результатах имитационного моделирования.
    """
    if not os.path.exists(SIM_FILE_DIR):
        os.makedirs(SIM_FILE_DIR)
    df.to_csv(SIM_FILE_PATH, index_label='Id')

    
def load_sim_data() -> pd.DataFrame:
    """
    Загрузить данные о резулдьтатах имитационного моделирования.
    """       
    if os.path.exists(SIM_FILE_PATH):
        return pd.read_csv(SIM_FILE_PATH, index_col='Id')
    return pd.DataFrame(columns=COLUMNS, index_col='Id')

sim_data = load_sim_data()
sim_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 150 entries, 0 to 149
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ArrAvg        150 non-null    float64
 1   ArrStd        150 non-null    float64
 2   ArrCv         150 non-null    float64
 3   SrvAvg        150 non-null    float64
 4   SrvStd        150 non-null    float64
 5   SrvCv         150 non-null    float64
 6   Rho           150 non-null    float64
 7   NetSize       150 non-null    float64
 8   Capacity      150 non-null    float64
 9   NumPackets    150 non-null    float64
 10  DelayAvg      150 non-null    float64
 11  DelayStd      150 non-null    float64
 12  DeliveryProb  150 non-null    float64
dtypes: float64(13)
memory usage: 16.4 KB


Функция `simulate(df, force=False, ...) -> pd.DataFrame` работает следующим образом:

- если в `df` нет строки, соответствующей переданным параметрам, то выполняется симуляция и функция возвращает новый `DataFrame`, содержащий результаты симуляции;
- если в `df` есть строка, соответствующая переданным параметрам, то симуляция выполняется, если выполнено любое из следующих условий:
    - передан аргумент `debug=True`
    - в настройках окружения (см. выше) установлен флаг `FORCE_SIMULATION=True`
    - если число пакетов, которые нужно промоделировать, больше числа пакетов, которое использовалось в предыдущей симуляции

Функция возвращает новый `DataFrame`, содержащий результаты заданной симуляции.

In [8]:
def simulate(
    df: pd.DataFrame, *, 
    arr_avg: float,
    arr_std: float,
    srv_avg: float,
    srv_std: float,
    net_size: int,
    capacity: int,
    num_packets: int,
    force: bool = False
) -> pd.DataFrame:
    """
    Выполнить симуляцию, если результатов нет в `df` или требуется их пересчитать, и вернуть новый `DataFrame`.
    """
    row_df = df[
        (df.ArrAvg == arr_avg) &
        (df.ArrStd == arr_std) &
        (df.SrvAvg == srv_avg) &
        (df.SrvStd == srv_std) &
        (df.NetSize == net_size) &
        (df.Capacity == capacity)]
    
    # Вычислим признаки, которые говорят о необходимости пересчета:
    no_row = len(row_df) == 0
    not_enough_packets = (not no_row) and (row_df.NumPackets.iloc[0] < num_packets)

    # Проверим, нужно ли пересчитать результаты:
    if force or no_row or not_enough_packets:
        arr = fit_ph(avg=arr_avg, std=arr_std)
        srv = fit_ph(avg=srv_avg, std=srv_std)
        ret = simulate_tandem(arr, [srv] * net_size, capacity, num_packets)

        row_data = {
            'ArrAvg': arr_avg,
            'ArrStd': arr_std,
            'ArrCv': arr_std / arr_avg,
            'SrvAvg': srv_avg,
            'SrvStd': srv_std,
            'SrvCv': srv_std / srv_avg,
            'Rho': srv_avg / arr_avg,
            'NetSize': net_size,
            'Capacity': capacity,
            'NumPackets': num_packets,
            'DelayAvg': ret.delivery_delays[0].avg,
            'DelayStd': ret.delivery_delays[0].std,
            'DeliveryProb': ret.delivery_prob[0],
        }

        # Если строки еще вообще не было, добавляем ее, а если была - обновляем:
        if no_row:
            df = df.append(row_data, ignore_index=True)
        else:
            df.update(pd.DataFrame(row_data, index=[row_df.index[0]]))

    return df

## Определяем сетку параметров модели

Будем считать, что сеть состоит из $L$ узлов, пакеты поступают на первую станцию и передаються по сети до тех пор, пока не будут обслужены последней станцией, либо не будут потерены из-за переполнения буфера на очередном узле.

Сеть будем описывать с помощью шести параметров:

- среднее значение интервалов между последовательными поступлениями пакетов в сеть ($\lambda^{-1}$)
- стандартное отклонение величин интервалов между последовательными поступлениями пакетов в сеть ($\sigma_A$)
- средняя длительность обслуживания ($\mu^{-1}$)
- стандартное отклонение длительности обслуживания ($\sigma_S$)
- число станций в сети ($L$)
- емкость очередей ($N$)

Из этих параметров можно полуить производные значения, которые оказываются более удобными при анализе:

- загрузка первой станции $\rho = \lambda / \mu$
- коэффициент вариации интервалов между поступлениями пакетов $c_A = \lambda \sigma_A$
- коэффициент вариации времени обслуживания $c_S = \mu \sigma_S$

Сетку будем задавать на множестве параметров $(\lambda^{-1}, \sigma_A, \mu^{-1}, \sigma_S, L, N)$.

Чтобы не пересчитывать результаты каждый раз заново, будем сохранять результаты расчета в файл `data/01_tandem_simulations.csv`. Если такого файла нет, или установлен флаг `FORCE_SIMULATION = True`, то каждая точка сетки будет рассчитана заново, а результаты расчета будут сохранены в файл. В противном случае расчет будет выполняться только в тогда, когда точки нет в файле, или в текущем расчете предполагается моделировать больше пакетов, то есть получить более точные результаты.

In [9]:
ARRIVAL_AVG = [1, 3, 5, 7, 10, 13, 15, 17]
ARRIVAL_STD = [1, 3, 5, 7, 10, 13, 15, 17]
SERVICE_AVG = [2.5, 3. , 5. , 5.5, 7. , 8]
SERVICE_STD = [1.,  2.5,  4, 5, 6, 7.5, 10]
NET_SIZE = [5,  7,  9, 11, 15]
CAPACITY = [6, 10, 12]

# Объединим все параметры в декартово произведение:
ALL_PARAMS = list(product(ARRIVAL_AVG, ARRIVAL_STD, SERVICE_AVG, SERVICE_STD, NET_SIZE, CAPACITY))

# Выполним симуляцию, если нужно:
for arr_avg, arr_std, srv_avg, srv_std, net_size, capacity in tqdm(ALL_PARAMS):
    sim_data = simulate(
        sim_data,
        arr_avg=arr_avg,
        arr_std=arr_std,
        srv_avg=srv_avg,
        srv_std=srv_std,
        net_size=net_size,
        capacity=capacity,
        num_packets=NUM_PACKETS,
        force=FORCE_SIMULATION
    )

print(sim_data.info())
print(sim_data)

# Сохраняем результат:
save_sim_data(sim_data)

  0%|          | 0/40320 [00:00<?, ?it/s]

KeyboardInterrupt: 